In [1]:
import opensim as osim
import os
import xml.etree.ElementTree as ET

action = "Shallow Squat"

osim.ModelVisualizer.addDirToGeometrySearchPaths("C:/OpenSim 4.5/Geometry")

base_path = f"C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/{action}"
model_path = "C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/Models"
ik_path = os.path.join(base_path, 'IK')
grf_path = os.path.join(base_path, 'ID', 'GRF')

model_file = 'Hanif-Scaled.osim'

if not os.path.exists(os.path.join(grf_path)):
  os.mkdir(grf_path)
    

ik_files = [f for f in os.listdir(ik_path) if f.endswith('.mot')]
grf_files = [f for f in os.listdir(grf_path) if f.endswith('.mot')]

print(f"Loading model from:", model_file)
print(f"IK files found {len(ik_files)}:", ik_files)
print(f"GRF files found {len(grf_files)}:", grf_files)


Loading model from: Hanif-Scaled.osim
IK files found 9: ['1_Hanif_Shallow_2_5_patched.mot', '1_Hanif_Shallow_5_0_patched.mot', '1_Hanif_Shallow_7_5_patched.mot', '2_Hanif_Shallow_2_5_patched.mot', '2_Hanif_Shallow_5_0_patched.mot', '2_Hanif_Shallow_7_5_patched.mot', '3_Hanif_Shallow_2_5_patched.mot', '3_Hanif_Shallow_5_0_patched.mot', '3_Hanif_Shallow_7_5_patched.mot']
GRF files found 9: ['1_Hanif_Shallow_2_5_grf.mot', '1_Hanif_Shallow_5_0_grf.mot', '1_Hanif_Shallow_7_5_grf.mot', '2_Hanif_Shallow_2_5_grf.mot', '2_Hanif_Shallow_5_0_grf.mot', '2_Hanif_Shallow_7_5_grf.mot', '3_Hanif_Shallow_2_5_grf.mot', '3_Hanif_Shallow_5_0_grf.mot', '3_Hanif_Shallow_7_5_grf.mot']


In [2]:
# Convert _grf.mot files to external loads .xml

template_xml = ".\Setup_XMLs\Boiler_XMLs\external_load_template.xml"

for grf_file in grf_files:
  tree = ET.parse(template_xml)
  for elem in tree.iter('datafile'):
      elem.text = os.path.join(grf_path, grf_file)

  output_xml = grf_file.replace('.mot', '.xml')
  tree.write(os.path.join(grf_path, output_xml), encoding="UTF-8", xml_declaration=True)

In [3]:
# Inverse Dynamics Batch Processing

for ik_file, grf_file in zip(ik_files, grf_files):
    grf_xml = grf_file.replace('.mot', '.xml')

    storage = osim.Storage(os.path.join(ik_path, ik_file))
    start_time = storage.getFirstTime()
    end_time = storage.getLastTime()

    print(f"IK File: {ik_file}, Start Time: {start_time}, End Time: {end_time}")

    id_tool = osim.InverseDynamicsTool()
    id_tool.setModelFileName(os.path.join(model_path, model_file))

    forces = osim.ArrayStr()
    forces.append("Muscles")
    id_tool.setExcludedForces(forces)

    id_tool.setCoordinatesFileName(os.path.join(ik_path, ik_file))
    id_tool.setExternalLoadsFileName(os.path.join(grf_path, grf_xml))
    id_tool.setLowpassCutoffFrequency(6.0)
    id_tool.setStartTime(start_time)
    id_tool.setEndTime(end_time)

    output_id_file = ik_file.replace('.mot', '.sto')
    id_tool.setResultsDir(os.path.join(base_path, 'ID'))
    id_tool.setOutputGenForceFileName(output_id_file)

    print(f"Processing ID for: {ik_file} with GRF: {grf_xml}")
    id_tool.run()

IK File: 1_Hanif_Shallow_2_5_patched.mot, Start Time: 0.2, End Time: 1.7
Processing ID for: 1_Hanif_Shallow_2_5_patched.mot with GRF: 1_Hanif_Shallow_2_5_grf.xml
IK File: 1_Hanif_Shallow_5_0_patched.mot, Start Time: 0.2, End Time: 2.2
Processing ID for: 1_Hanif_Shallow_5_0_patched.mot with GRF: 1_Hanif_Shallow_5_0_grf.xml
IK File: 1_Hanif_Shallow_7_5_patched.mot, Start Time: 0.8, End Time: 3.1
Processing ID for: 1_Hanif_Shallow_7_5_patched.mot with GRF: 1_Hanif_Shallow_7_5_grf.xml
IK File: 2_Hanif_Shallow_2_5_patched.mot, Start Time: 2.2, End Time: 3.7
Processing ID for: 2_Hanif_Shallow_2_5_patched.mot with GRF: 2_Hanif_Shallow_2_5_grf.xml
IK File: 2_Hanif_Shallow_5_0_patched.mot, Start Time: 2.3, End Time: 4.3
Processing ID for: 2_Hanif_Shallow_5_0_patched.mot with GRF: 2_Hanif_Shallow_5_0_grf.xml
IK File: 2_Hanif_Shallow_7_5_patched.mot, Start Time: 3.4, End Time: 5.8
Processing ID for: 2_Hanif_Shallow_7_5_patched.mot with GRF: 2_Hanif_Shallow_7_5_grf.xml
IK File: 3_Hanif_Shallow_2_5